Before this, you should have downloaded and unzipped all the data from kaggle.
Be sure to update your working directory to get the data all formatted.

This is directly from https://www.kaggle.com/yhn112/resnet18-baseline-pytorch-ignite/notebook

In [1]:
import numpy as np
import pandas as pd

from PIL import Image

import torch
import torch.nn as nn
import torch.utils.data as D
import torch.nn.functional as F

import torchvision
from torchvision import transforms as T
from torchvision import models

import tqdm

from sklearn.model_selection import train_test_split

from PIL import Image
import matplotlib.pyplot as plt

Be sure to update path_data

In [2]:
path_data = 'D:\\Data\\DeepLearning\\recursion-cellular-image-classification'
batch_size = 32
torch.manual_seed(0)

In [59]:
class ImagesDS(D.Dataset):
    def __init__(self, df, img_dir, mode='train', site=1, channels=[1,2,3,4,5,6], transform=None):
        self.records = df.to_records(index=False)
        self.channels = channels
        # self.site = site
        self.mode = mode
        self.img_dir = img_dir
        self.len = df.shape[0]
        self.transform = transform
        unique_list = np.unique([int(n.split("_")[1]) for n in df['sirna']])
        self.mapping = {}
        for (i, val) in enumerate(unique_list):
            self.mapping[val] = i


    @staticmethod
    def _load_img_as_tensor(file_name):
        with Image.open(file_name) as img:
            return T.ToTensor()(img)

    def _get_img_path(self, index, channel, site):
        mode = self.mode
        if self.mode == 'valid':
            mode = 'train'
        experiment, well, plate = self.records[index].experiment, self.records[index].well, self.records[index].plate
        return '/'.join([self.img_dir,mode,experiment,f'Plate{plate}',f'{well}_s{site}_w{channel}.png'])

    def __getitem__(self, index):
        paths_1 = [self._get_img_path(index, ch, 1) for ch in self.channels]
        paths_2 = [self._get_img_path(index, ch, 2) for ch in self.channels]
        img_1 = torch.cat([self._load_img_as_tensor(img_path) for img_path in paths_1])
        img_2 = torch.cat([self._load_img_as_tensor(img_path) for img_path in paths_2])
        img = torch.cat([img_1, img_2])
        if self.mode == 'train':
            # if training, then apply transformation
            if self.transform is not None:
                img = self.transform(img)
            return img, self.mapping[int(self.records[index].sirna.split("_")[1])]
        elif self.mode == 'valid':
            return img, self.mapping[int(self.records[index].sirna.split('_')[1])]
        else:
            return img, self.records[index].id_code

    def __len__(self):
        return self.len

Define transformations

In [60]:
transforms = T.Compose([
    T.RandomHorizontalFlip(),
    T.RandomRotation(90),
    T.RandomVerticalFlip()
])

# transforms = T.Compose([
#     T.Resize((224,224)),
#     T.CenterCrop(224),
#     # T.ToTensor(),
#     T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#     T.RandomHorizontalFlip(),
#     T.RandomRotation(90),
#     T.RandomVerticalFlip()
# ])

Gets data from site 1

In [65]:
df = pd.read_csv(path_data+'/train.csv')
df_train, df_test = train_test_split(df, test_size = 0.1, random_state=42)
df_train, df_val = train_test_split(df_train, test_size=.01, random_state=42)
# df_test = pd.read_csv(path_data+'/test.csv')

In [66]:
ds_train = ImagesDS(df_train, path_data, mode='train', transform=transforms)
ds_val = ImagesDS(df_val, path_data, mode='valid')
ds_test = ImagesDS(df_test, path_data, mode='test')

{0: 0,
 1: 1,
 2: 2,
 3: 3,
 4: 4,
 5: 5,
 6: 6,
 7: 7,
 8: 8,
 9: 9,
 10: 10,
 11: 11,
 12: 12,
 13: 13,
 14: 14,
 15: 15,
 16: 16,
 17: 17,
 18: 18,
 19: 19,
 20: 20,
 21: 21,
 22: 22,
 23: 23,
 24: 24,
 25: 25,
 26: 26,
 27: 27,
 28: 28,
 29: 29,
 30: 30,
 31: 31,
 32: 32,
 33: 33,
 34: 34,
 35: 35,
 36: 36,
 37: 37,
 38: 38,
 39: 39,
 40: 40,
 41: 41,
 42: 42,
 43: 43,
 44: 44,
 45: 45,
 46: 46,
 47: 47,
 48: 48,
 49: 49,
 50: 50,
 51: 51,
 53: 52,
 54: 53,
 55: 54,
 56: 55,
 57: 56,
 58: 57,
 59: 58,
 60: 59,
 61: 60,
 62: 61,
 63: 62,
 64: 63,
 65: 64,
 66: 65,
 67: 66,
 68: 67,
 69: 68,
 70: 69,
 71: 70,
 72: 71,
 73: 72,
 74: 73,
 75: 74,
 76: 75,
 77: 76,
 78: 77,
 79: 78,
 80: 79,
 81: 80,
 82: 81,
 83: 82,
 84: 83,
 85: 84,
 86: 85,
 87: 86,
 88: 87,
 89: 88,
 90: 89,
 91: 90,
 92: 91,
 93: 92,
 94: 93,
 95: 94,
 96: 95,
 97: 96,
 98: 97,
 99: 98,
 100: 99,
 101: 100,
 102: 101,
 103: 102,
 104: 103,
 105: 104,
 106: 105,
 107: 106,
 108: 107,
 109: 108,
 110: 109,
 111: 110

Creates DataLoader which is what is used for the models

In [67]:
train_loader = D.DataLoader(ds_train, batch_size=batch_size, shuffle=True)
test_loader = D.DataLoader(ds_test, batch_size=batch_size, shuffle=False)
valid_loader = D.DataLoader(ds_val, batch_size=batch_size, shuffle=False)

KeyboardInterrupt: 

# Example model training

In [53]:
device='cpu'

classes = 1108
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, classes)

trained_kernel = model.conv1.weight
new_conv = nn.Conv2d(12, 64, 7, 2, 3, bias=False)
with torch.no_grad():
    new_conv.weight[:,:] = torch.stack([torch.mean(trained_kernel, 1)] * 12, dim=1)
model.conv1 = new_conv

In [54]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)

In [68]:
def validation_accuracy(model, testloader, p=False):
    with torch.no_grad(): # In test phase, we don't need to compute gradients (for memory efficiency)
        correct = 0
        total = 0
        for images, labels in testloader:

            '''Step - Move images to device after appropriate reshaping'''
            images = images.to(device)
            '''Step  - Move labels to device'''
            labels = labels.to(device)
            #get network outputs
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        if p:
            print(f'Accuracy of the network on the 10000 test images: {(100 * correct / total)}')
        return 100 * correct / total

In [57]:


Nepochs = 1
Nbatch = 20

stop = False
minimum_loss = .8

model.train()
model.to(device)
for i in range(Nepochs):
    if stop:
        break
    for j, batch in enumerate(train_loader):
        print(j)
        images, labels = batch
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        print(loss.item())
        if (j+1) % 10 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(i+1, Nepochs, j+1, len(train_loader), loss.item()))

        Nbatch -= 1
        if Nbatch <= 0 or loss <= minimum_loss:
            stop = True
            break

0
<built-in method item of Tensor object at 0x0000018082B539F0>
1
<built-in method item of Tensor object at 0x0000018082B484A0>
2
<built-in method item of Tensor object at 0x0000018082B483B0>
3
<built-in method item of Tensor object at 0x0000018082B48C20>
4
<built-in method item of Tensor object at 0x0000018082B489F0>
5
<built-in method item of Tensor object at 0x0000018082B48900>
6
<built-in method item of Tensor object at 0x0000018082B480E0>
7
<built-in method item of Tensor object at 0x0000018082B48860>
8
<built-in method item of Tensor object at 0x0000018082B48D10>
9
<built-in method item of Tensor object at 0x0000018082B486D0>
Epoch [1/1], Step [10/925], Loss: 7.4407
10
<built-in method item of Tensor object at 0x0000018082B489F0>
11
<built-in method item of Tensor object at 0x0000018082B484F0>
12
<built-in method item of Tensor object at 0x0000018082B48040>
13
<built-in method item of Tensor object at 0x0000018082B48C20>
14
<built-in method item of Tensor object at 0x0000018082B4

In [69]:
validation_accuracy(model, valid_loader, True)

Accuracy of the network on the 10000 test images: 0.303951367781155


0.303951367781155